# Data integration and doublet removal

In [ ]:
library(dplyr)
library(Seurat)
library(Matrix)
library(harmony) # batch effect removal
library(stringr)
library(SingleCellExperiment)

In [ ]:
library(SingleR) # cell type prediction
library(scds) # doublet removal

In [ ]:
# Remove the mitochondrial genes and ribosomal genes
remove_some_genes <- function(pbmc, genes=c("^RPS","^RPL","^MT-","MALAT1","NEAT")) {
    pbmc_sce <- as.SingleCellExperiment(pbmc)

    for (gene in genes){
        counts(pbmc_sce)[rownames(pbmc_sce)[grepl(gene, rownames(pbmc_sce))],] = 0
    }

    return (as.Seurat(pbmc_sce, counts = "counts"))
}

prep_seurat <- function(pbmc, percent.mito=40){ # If you include cancer cells, percent.mt = 40. Otherwise, percent.mt = 25
    pbmc[["percent.mt"]] <- PercentageFeatureSet(pbmc, pattern = "^MT-")
    pbmc <- subset(pbmc, subset = nFeature_RNA > 150 & percent.mt < percent.mito)
    
    pbmc <- remove_some_genes(pbmc)
    
    pbmc <- NormalizeData(pbmc)
    # store mitochondrial percentage in object meta data
#    pbmc <- PercentageFeatureSet(pbmc, pattern = "^MT-", col.name = "percent.mt")
    # run sctransform
#    pbmc <- SCTransform(pbmc, vars.to.regress = "percent.mt", verbose = FALSE)
    
    return(pbmc)
    }

## data integration

In [ ]:
data.list <- c()

In [ ]:
infiles = Sys.glob("/data/share/scRNAseq/results/Stanford_human_STAD/soupx/*.Rds")
samples = c()
data = list()
for (infile in infiles){
    sample = basename(infile)[7]
    sample = str_replace(sample, "Stanford_","")
    sample = str_replace(sample, "-total.Rds","")
    samples = c(samples, sample)
    load(infile)
    data <- c(data, prep_seurat(sobj))
}

indirs = Sys.glob("/data/share/scRNAseq/results/human_STAD/cellranger_3.1/*-total/outs")
indirs = indirs[c(-2,-16)] #2:10T-total, 16:18N-total
indirs2 = Sys.glob("/data/share/scRNAseq/results/human_STAD/cellranger_3.1/*-Bcell/outs")
indirs3 = Sys.glob("/data/share/scRNAseq/results/human_STAD/cellranger_3.1/*-GEX/outs")
indirs = c(indirs, indirs2, indirs3)

for (indir in indirs){
    sample = str_split(indir, "/", simplify = TRUE)[8]
    sample = str_replace(sample, "-GEX","")
    samples = c(samples, sample)
    outfile = paste0("/data/share/scRNAseq/results/human_STAD/cellranger_3.1/soupx/",sample,".Rds")
    load(outfile)
    data <- c(data, prep_seurat(sobj))
}

In [6]:
indirs = Sys.glob("/data/share/scRNAseq/results/Tsinghua_STAD_fastq_soupx/")

In [7]:
for (file in list.files(indirs)[-1]){ # /2/は無視
    load(paste0(indirs,file))
    data <- c(data, prep_seurat(sobj))
}

In [10]:
for (i in 1:length(data)){
    samples[i] <- as.character(data[[i]]@meta.data$orig.ident[1])
}

### merge

In [15]:
data.all <- merge(data[1][[1]], 
                  y = data[2:length(data)], 
                  add.cell.ids = samples, 
                  project = "all_data", 
                  merge.data = TRUE)

In [16]:
pbmc <- FindVariableFeatures(data.all, selection.method = "vst", nfeatures = 4000)
var.genes <- pbmc@assays$RNA@var.features
all.genes <- rownames(pbmc)
pbmc <- ScaleData(pbmc, features = var.genes)#, vars.to.regress="percent.mt")
pbmc <- RunPCA(pbmc, verbose = FALSE)

Centering and scaling data matrix



### batch effect removal

In [ ]:
pbmc <- pbmc %>% 
    RunHarmony("orig.ident", plot_convergence = FALSE) %>% 
    RunUMAP(reduction = "harmony", dims = 1:40) %>% 
    FindNeighbors(reduction = "harmony", dims = 1:40) %>% 
    FindClusters(resolution = 0.5) %>% 
    identity()

Warning message:
“Quick-TRANSfer stage steps exceeded maximum (= 20141500)”
Warning message:
“Quick-TRANSfer stage steps exceeded maximum (= 20141500)”
Warning message:
“Quick-TRANSfer stage steps exceeded maximum (= 20141500)”
Warning message:
“Quick-TRANSfer stage steps exceeded maximum (= 20141500)”
Warning message:
“Quick-TRANSfer stage steps exceeded maximum (= 20141500)”
Warning message:
“Quick-TRANSfer stage steps exceeded maximum (= 20141500)”
Warning message:
“Quick-TRANSfer stage steps exceeded maximum (= 20141500)”
Warning message:
“Quick-TRANSfer stage steps exceeded maximum (= 20141500)”
Harmony 1/10

Harmony 2/10

Harmony 3/10

Harmony 4/10

Harmony 5/10

Harmony converged after 5 iterations

Warning message:
“Invalid name supplied, making object name syntactically valid. New object name is Seurat..ProjectDim.RNA.harmony; see ?make.names for more details on syntax validity”
16:17:36 UMAP embedding parameters a = 0.9922 b = 1.112

16:17:37 Read 402830 rows and found 40 num

### Confirm the harmony effect

In [ ]:
DimPlot(pbmc, reduction = "umap", label = TRUE, pt.size = .1)

In [ ]:
DimPlot(pbmc, reduction = "umap", group.by = "orig.ident", pt.size = .1, split.by = 'orig.ident',ncol = 4)

### add metadata 

In [ ]:
pat = as.character(pbmc[['orig.ident']]$orig.ident)
pat = str_replace(pat, pattern="Tsinghua_12320046", replacement="ECG")
pat = str_replace(pat, pattern="Tsinghua_12320047M", replacement="IMS1")
pat = str_replace(pat, pattern="Tsinghua_12320048M", replacement="IMS2")
pat = str_replace(pat, pattern="Tsinghua_12320049M", replacement="IMS3")
pat = str_replace(pat, pattern="Tsinghua_12320050M", replacement="IMS4")
pat = str_replace(pat, pattern="Tsinghua_12320051M", replacement="IMW1")
pat = str_replace(pat, pattern="Tsinghua_12320052M", replacement="IMW2")
pat = str_replace(pat, pattern="Tsinghua_12320054M", replacement="CAG1")
pat = str_replace(pat, pattern="Tsinghua_12320055M", replacement="CAG2")
pat = str_replace(pat, pattern="Tsinghua_12320056M", replacement="NAG1")
pat = str_replace(pat, pattern="Tsinghua_12320057M", replacement="NAG2")
pat = str_replace(pat, pattern="Tsinghua_12320058M", replacement="NAG3")

pat = str_replace(pat, pattern="N-total", replacement="")
pat = str_replace(pat, pattern="N-Bcell", replacement="")
pat = str_replace(pat, pattern="T-total", replacement="")
pat = str_replace(pat, pattern="T-Bcell", replacement="")
pat = str_replace(pat, pattern="T1-total", replacement="")
pat = str_replace(pat, pattern="T2-total", replacement="")
pat = str_replace(pat, pattern="T1-Bcell", replacement="")
pat = str_replace(pat, pattern="T2-Bcell", replacement="")
pat = str_replace(pat, pattern="Stanford_", replacement="")
pat = str_replace(pat, pattern="N1", replacement="")
pat = str_replace(pat, pattern="T1", replacement="")
pat = str_replace(pat, pattern="N2", replacement="")
pat = str_replace(pat, pattern="T2", replacement="")
pat = str_replace(pat, pattern="PBMC", replacement="")
pat = str_replace(pat, pattern="-total", replacement="")

In [ ]:
TN = as.character(pbmc[['orig.ident']]$orig.ident)

TN = str_replace(TN, pattern="Tsinghua_12320046", replacement="T")
TN = str_replace(TN, pattern="Tsinghua_12320047M", replacement="IM")
TN = str_replace(TN, pattern="Tsinghua_12320048M", replacement="IM")
TN = str_replace(TN, pattern="Tsinghua_12320049M", replacement="IM")
TN = str_replace(TN, pattern="Tsinghua_12320050M", replacement="IM")
TN = str_replace(TN, pattern="Tsinghua_12320051M", replacement="IM")
TN = str_replace(TN, pattern="Tsinghua_12320052M", replacement="IM")
TN = str_replace(TN, pattern="Tsinghua_12320054M", replacement="N")
TN = str_replace(TN, pattern="Tsinghua_12320055M", replacement="N")
TN = str_replace(TN, pattern="Tsinghua_12320056M", replacement="N")
TN = str_replace(TN, pattern="Tsinghua_12320057M", replacement="N")
TN = str_replace(TN, pattern="Tsinghua_12320058M", replacement="N")

TN = str_replace(TN, pattern="Stanford_", replacement="")
TN = str_replace(TN, pattern="-total", replacement="")
TN = str_replace(TN, pattern="6649N1", replacement="IM")
TN = str_replace(TN, pattern="6649T1", replacement="IM")
TN = str_replace(TN, pattern="^.*15T", replacement="T") 
TN = str_replace(TN, pattern="^.*5T", replacement="N") #sample 5 はnormalだった
TN = str_replace(TN, pattern="^.*N", replacement="N")
TN = str_replace(TN, pattern="^.*T", replacement="T")
TN = str_replace(TN, pattern="^.*PBMC", replacement="PBMC")
TN = str_replace(TN, pattern="1", replacement="")
TN = str_replace(TN, pattern="2", replacement="")
TN = str_replace(TN, pattern="^.*N", replacement="N")
TN = str_replace(TN, pattern="^.*T", replacement="T")
TN = str_replace(TN, pattern="^.*T1", replacement="T")
TN = str_replace(TN, pattern="^.*T2", replacement="T")
TN = str_replace(TN, pattern="-.*$", replacement="")

In [ ]:
rep = as.character(pbmc[['orig.ident']]$orig.ident)

rep = str_replace(rep, pattern="Tsinghua_12320046", replacement="1")
rep = str_replace(rep, pattern="Tsinghua_12320047M", replacement="1")
rep = str_replace(rep, pattern="Tsinghua_12320048M", replacement="1")
rep = str_replace(rep, pattern="Tsinghua_12320049M", replacement="1")
rep = str_replace(rep, pattern="Tsinghua_12320050M", replacement="1")
rep = str_replace(rep, pattern="Tsinghua_12320051M", replacement="1")
rep = str_replace(rep, pattern="Tsinghua_12320052M", replacement="1")
rep = str_replace(rep, pattern="Tsinghua_12320054M", replacement="1")
rep = str_replace(rep, pattern="Tsinghua_12320055M", replacement="1")
rep = str_replace(rep, pattern="Tsinghua_12320056M", replacement="1")
rep = str_replace(rep, pattern="Tsinghua_12320057M", replacement="1")
rep = str_replace(rep, pattern="Tsinghua_12320058M", replacement="1")

rep = str_replace(rep, pattern="^.*N", replacement="1")
rep = str_replace(rep, pattern="^5T", replacement="2")
rep = str_replace(rep, pattern="11-total", replacement="1")
rep = str_replace(rep, pattern="12-total", replacement="1")
rep = str_replace(rep, pattern="^.*T1", replacement="1")
rep = str_replace(rep, pattern="^.*T2", replacement="2")
rep = str_replace(rep, pattern="^.*T", replacement="1")
rep = str_replace(rep, pattern="-.*$", replacement="")
rep = str_replace(rep, pattern="Stanford_5931PBMC", replacement="1")
rep = str_replace(rep, pattern="Stanford_6207PBMC", replacement="1")

In [ ]:
subtype = pat

subtype = str_replace(subtype, pattern='CAG.', replacement='CAG')
subtype = str_replace(subtype, pattern='NAG.', replacement='NAG')
subtype = str_replace(subtype, pattern='IM.', replacement='Metaplasia')

subtype = str_replace(subtype, pattern="5846", replacement="DGC")
subtype = str_replace(subtype, pattern="5866", replacement="DGC")
subtype = str_replace(subtype, pattern="5931", replacement="MSI")
subtype = str_replace(subtype, pattern="6207", replacement="MSI")
subtype = str_replace(subtype, pattern="6342", replacement="DGC")
subtype = str_replace(subtype, pattern="6592", replacement="MSI")
subtype = str_replace(subtype, pattern="6649", replacement="Metaplasia")
subtype = str_replace(subtype, pattern="6709", replacement="MSI")
subtype = str_replace(subtype, pattern="18", replacement="DGC")
subtype = str_replace(subtype, pattern="17", replacement="DGC")
subtype = str_replace(subtype, pattern="16", replacement="IGC")
subtype = str_replace(subtype, pattern="15", replacement="AFP")
subtype = str_replace(subtype, pattern="14", replacement="IGC")
subtype = str_replace(subtype, pattern="12", replacement="DGC")
subtype = str_replace(subtype, pattern="11", replacement="MSI")
subtype = str_replace(subtype, pattern="10", replacement="IGC")
subtype = str_replace(subtype, pattern="9", replacement="MSI")
subtype = str_replace(subtype, pattern="8", replacement="DGC")
subtype = str_replace(subtype, pattern="7", replacement="DGC")
subtype = str_replace(subtype, pattern="6", replacement="IGC")
subtype = str_replace(subtype, pattern="5", replacement="Unknown(Normal)")
subtype = str_replace(subtype, pattern="4", replacement="DGC")
subtype = str_replace(subtype, pattern="3", replacement="DGC")
subtype = str_replace(subtype, pattern="2", replacement="ASC")
subtype = str_replace(subtype, pattern="1", replacement="AFP")

In [ ]:
pbmc[['TN']] = TN
pbmc[['patient']] = pat
pbmc[['replicate']] = rep
pbmc[['subtype']] = subtype

## cell type prediction using SingleR

In [ ]:
# SingleR reference data
hpca.se <- HumanPrimaryCellAtlasData()
#hpca.se

In [ ]:
# SingleR
# take some time
pbmc <- NormalizeData(pbmc, block.size=20)

In [ ]:
pred.pbmc <- SingleR(test = pbmc@assays$RNA@data, 
                         ref=hpca.se,
                        labels = hpca.se$label.main)

In [ ]:
pbmc$celltype_singleR = pred.pbmc$labels

In [49]:
# take a long time
# doublet pridiction
data <- c()
for (i in 1:length(unique(pbmc$orig.ident))){
pbmc2 <- pbmc[,pbmc$orig.ident==unique(pbmc$orig.ident)[i]]
sce <- as.SingleCellExperiment(pbmc2)
logcounts(sce) = log1p(counts(sce))
vrs            = apply(logcounts(sce),1,var)
sce = cxds(sce,retRes = TRUE)            #find doublets/multiples in UMI scRNS-seq data
sce = bcds(sce,retRes = TRUE,verb=TRUE)  #find doublets/multiples in UMI scRNS-seq data
sce = cxds_bcds_hybrid(sce)
pbmc2 = as.Seurat(sce)
data <- c(data, pbmc2)
}

-> selecting genes


-> simulating doublets


-> training classifier


-> done.



Warning message:
“Keys should be one or more alphanumeric characters followed by an underscore, setting key from PC__ to PC_”
Warning message:
“All keys should be one or more alphanumeric characters followed by an underscore '_', setting key to PC_”
Warning message:
“Keys should be one or more alphanumeric characters followed by an underscore, setting key from harmony__ to harmony_”
Warning message:
“All keys should be one or more alphanumeric characters followed by an underscore '_', setting key to harmony_”
Warning message:
“Keys should be one or more alphanumeric characters followed by an underscore, setting key from UMAP__ to UMAP_”
Warning message:
“All keys should be one or more alphanumeric characters followed by an underscore '_', setting key to UMAP_”
-> selecting genes


-> simulating doublets


-> training classifier


-> done.



Warning message:
“Keys should be one or more alphanumeric chara

In [85]:
data_all <- merge(data[1][[1]], 
                  y = data[2:length(data)],  
                  project = "all_data", 
                  merge.data = TRUE)

In [87]:
pbmc3 <- data_all

In [88]:
# filter the low quality cells
# all cells: scds hybdrid_socre < 1 &&
# epithelial cells: percent.mt>25
# others: percent.mt > 15
pbmc3$cells.use = pbmc3$hybrid_score < 1 
pbmc3$cells.use[(pbmc3$celltype_singleR %in% c("Epithelial_cells", "Hepatocytes")) & (pbmc3$percent.mt > 25) & (pbmc3$TN == 'N') ] = FALSE # 正常上皮は percent.mt > 25を除く
pbmc3$cells.use[(pbmc3$celltype_singleR %in% c("Epithelial_cells", "Hepatocytes")) & (pbmc3$percent.mt > 25) & (pbmc3$TN == 'IM') ] = FALSE # 正常上皮は percent.mt > 25を除く
pbmc3$cells.use[!(pbmc3$celltype_singleR %in% c("Epithelial_cells", "Hepatocytes")) & (pbmc3$percent.mt > 15) ] = FALSE # 上皮以外は percent.mt > 15を除く

In [89]:
pbmc <- subset(pbmc3, cells.use == TRUE) #hybrid filter

### remove barcode swapping ID (check how to find swapping ID in other section)

In [92]:
swapping <- read.table('/data/tsubosaka/data/swap_barcode.txt')
swapping <- as.character(swapping$V1)

In [96]:
Idents(pbmc) <- 'TN'
pbmc_normal <- subset(pbmc, idents=c('N','IM'))

In [101]:
saveRDS(pbmc, '/home/tsubosaka/integrated_data/論文用再検討/0_integrated_data_all.rds')

In [102]:
saveRDS(pbmc_normal, '/home/tsubosaka/integrated_data/論文用再検討/0_integrated_data_normal.rds')

### done